Hunter Bennett | Glass Lab | Kupffer Strains Project | 19 April 2021

In addition to calling differential peaks we want to examine the mutational burden in enhancers with different log2fc between strains (similar to analysis done in our other strains papers)

In [13]:
### header ###
__author__ = "Hunter Bennett"
__license__ = "BSD"
__email__ = "hunter.r.bennett@gmail.com"
%load_ext autoreload
%autoreload 2
### imports ###
import sys
%matplotlib inline
import os
import re
import glob
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 200
sns.set(font_scale=1)
sns.set_context('talk')
sns.set_style('white')

# import custom functions
import sys
sys.path.insert(0, '/home/h1bennet/code/')
from hbUtils import ngs_qc, quantile_normalize_df
from plotting_scripts import label_point, pca_rpkm_mat, get_diff_volcano
from homer_preprocessing import read_annotated_peaks, import_homer_diffpeak, pull_comparisons_get_diff

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
dataDirectory = ''
workingDirectory = '/home/h1bennet/strains/results/06b_Strains_Control_Combined_H3K27Ac/'
if not os.path.isdir(workingDirectory):
    os.mkdir(workingDirectory)
os.chdir(workingDirectory)

In [15]:
best_reps = ['/home/h1bennet/strains/data/H3K27Ac/control/02_aj_Kupffer_H3K27Ac_control_young_AJ10ABC_161208',
             '/home/h1bennet/strains/data/H3K27Ac/control/02_aj_Kupffer_H3K27Ac_control_young_AJ11AB_161208',
             '/home/h1bennet/strains/data/H3K27Ac/control_cohort2_untrimmed/tag_mouse_aj_Male_Kupffer_ChIP_H3K27ac_Chow_healthyoung_AJ1104_TDT_l20201212_GGCTTAAG_TCGTGACC_S29_L001.aj.bowtie2_shifted_from_AJ.sam',
             '/home/h1bennet/strains/data/H3K27Ac/control/01_balbc_Kupffer_H3K27Ac_control_young_Balb10ABC_170915',
             '/home/h1bennet/strains/data/H3K27Ac/control/01_balbc_Kupffer_H3K27Ac_control_young_Balb11AB_170915',
             '/home/h1bennet/strains/data/H3K27Ac/control_cohort2_untrimmed/tag_mouse_balbcj_Male_Kupffer_ChIP_H3K27ac_Chow_healthyoung_BALB1104_TDT_l20201212_AATCCGGA_CTACAGTT_S30_L001.balbcj.bowtie2_shifted_from_BALBCJ.sam',
             '/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915',
             '/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423',
             '/home/h1bennet/strains/data/H3K27Ac/control_cohort2_untrimmed/tag_mouse_c57bl6j_Male_Kupffer_ChIP_H3K27ac_Chow_healthyoung_C571104_TDT_l20201212_TAATACAG_ATATTCAC_S31_L001.c57bl6j.bowtie2_shifted_from_C57BL6J.sam']

# Save differential peaks for MMARGE mutational burden analysis.

In [16]:
diff_peak, peaks, peak_mat, peak_mat_quant = import_homer_diffpeak(
    './merged_peaks/diff_output.txt',
    './merged_peaks/ann_norm_kc_control_atac_peaks_all.txt')

annotatePeaks all peaks (86301, 27)
getDiffExpression selected transcripts (84264, 36)
annotatePeaks selected peaks (84264, 27)


In [17]:
# pull out selected samples
cols = np.append(np.asarray([True]*18, 'bool'), peak_mat.columns.str.contains('|'.join(best_reps)))
tst = np.append(np.asarray([True]*18, 'bool'), peak_mat.columns.str.contains('|'.join(best_reps)))
peaks = peaks.loc[:, tst]
peak_mat = peak_mat.loc[:, peak_mat.columns.str.contains('|'.join(best_reps))]

In [18]:
comp_dict = pull_comparisons_get_diff(diff_peak, seq_type='Peak')

In [19]:
comp_dict.keys()

dict_keys(['aj vs. balbcj', 'aj vs. c57bl6j', 'balbcj vs. c57bl6j'])

In [20]:
if not os.path.isdir('./marge_mutational_burden/'):
    os.mkdir('./marge_mutational_burden/')

In [21]:
pval = 0.05
fcs = [1,2,4]
peak_list_dict = {}

In [22]:
for fc in fcs:   
    for key in comp_dict.keys():
        # select get diff data frame.
        df = comp_dict[key]
        deg = df.index[(df.loc[:, 'adj_pval'] < pval) & (np.abs(df.loc[:, 'log2fc']) >= fc)]
        peak_list_dict[key+'_de_peaks_fc_'+str(fc)] = deg
        
        # add list for nonsig peaks
        nondeg = df.index[(df.loc[:, 'adj_pval'] >= pval) & (np.abs(df.loc[:, 'log2fc']) < fc)]
        peak_list_dict[key+'_nonsig'] = nondeg

### Re-create 200bp ATAC-seq peaks

In [26]:
atac_peaks = pd.read_csv('./merged_peaks/idr_peaks_merged.txt', sep='\t', index_col=0)

In [27]:
new_start = []
new_end = []
for index, row in atac_peaks.iterrows():
    center = row.start + int(np.floor((row.end - row.start)/2))
    new_start.append(center - 100)
    new_end.append(center + 100)
    
atac_peaks_200bp = atac_peaks.iloc[:, :4].copy(deep=True)
atac_peaks_200bp['start'] = new_start
atac_peaks_200bp['end'] = new_end
atac_peaks_200bp.index.rename('PeakID', inplace=True)

In [28]:
diff_peak_distal = diff_peak.loc[np.abs(diff_peak.loc[:, 'Distance to TSS'])>3000, :]
atac_peaks_200bp_distal = atac_peaks_200bp.reindex(diff_peak_distal.index).dropna(how='all')

# print to check that this worked
print(diff_peak.shape[0], 'peaks in differential peak file')
print(atac_peaks_200bp.shape[0], 'peaks in ATAC-seq file')
print()
print(diff_peak_distal.shape[0], 'distal peaks in differential peak file')
print(atac_peaks_200bp_distal.shape[0], 'distal peaks in ATAC-seq file')

84264 peaks in differential peak file
86301 peaks in ATAC-seq file

56602 distal peaks in differential peak file
56602 distal peaks in ATAC-seq file


if np.NaN get introduced into the matrix then it converts 'int' type columns to 'float' type columns, this is not ideal and interferes with downstream peak analysis so we create a dict to change the start and end columns back to integers just in case

In [33]:
convert_dict = {'start': int,
                'end': int}

for key in peak_list_dict.keys():
    # save cell specific promoters
    tmp = atac_peaks_200bp.reindex(peak_list_dict[key]).dropna(how='all')
    tmp = tmp.astype(convert_dict)
    tmp.to_csv('./marge_mutational_burden/'+key.replace(' vs. ','_vs_')+'.txt',
               sep='\t')

Annoate mutations with MMARGE

In [40]:
with open('./marge_annotate_mutations.sh', 'w') as f:
    for peakfile in glob.glob('./marge_mutational_burden/aj*c57*.txt'):
        anno_mut = ['MMARGE.pl annotate_mutations', '-file', peakfile,
                    '-output', peakfile.replace('.txt', '_anno_muts.txt'),
                    '-ind', 'aj\n\n']
        f.write(' '.join(anno_mut))
        
    for peakfile in glob.glob('./marge_mutational_burden/balbcj*c57*.txt'):
        anno_mut = ['MMARGE.pl annotate_mutations', '-file', peakfile,
                    '-output', peakfile.replace('.txt', '_anno_muts.txt'),
                    '-ind', 'balbcj\n\n']
        f.write(' '.join(anno_mut))
        
    for peakfile in glob.glob('./marge_mutational_burden/aj*balbcj*.txt'):
        anno_mut = ['MMARGE.pl annotate_mutations', '-file', peakfile,
                    '-output', peakfile.replace('.txt', '_anno_muts.txt'),
                    '-ind', 'balbcj\n\n']
        f.write(' '.join(anno_mut))

Calculate number of mutations in each file

In [41]:
np.sort(glob.glob('./marge_mutational_burden/*anno_muts.txt'))

array(['./marge_mutational_burden/aj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt'],
      dtype='<U67')

In [42]:
help(round)

Help on built-in function round in module builtins:

round(number, ndigits=None)
    Round a number to a given precision in decimal digits.
    
    The return value is an integer if ndigits is omitted or None.  Otherwise
    the return value has the same type as the number.  ndigits may be negative.



In [45]:
mut_frac_dict = {}
for peakfile in np.sort(glob.glob('./marge_mutational_burden/*anno_muts.txt')):
    mutfile = pd.read_csv(peakfile, index_col=0, sep='\t')
     #print(mutfile.iloc[:, -1].isna().value_counts())
    nomut = mutfile.iloc[:, -1].isna().sum()
    total_peaks = mutfile.iloc[:, -1].shape[0]
    mut_frac_dict[peakfile.split('/')[-1]] = round((total_peaks-nomut)/(total_peaks),
                                                   ndigits=3)

In [46]:
pd.Series(mut_frac_dict)

aj_vs_balbcj_de_peaks_fc_1_anno_muts.txt         0.275
aj_vs_balbcj_de_peaks_fc_2_anno_muts.txt         0.322
aj_vs_balbcj_de_peaks_fc_4_anno_muts.txt         0.281
aj_vs_balbcj_nonsig_anno_muts.txt                0.177
aj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt        0.555
aj_vs_c57bl6j_de_peaks_fc_2_anno_muts.txt        0.624
aj_vs_c57bl6j_de_peaks_fc_4_anno_muts.txt        0.611
aj_vs_c57bl6j_nonsig_anno_muts.txt               0.179
balbcj_vs_c57bl6j_de_peaks_fc_1_anno_muts.txt    0.456
balbcj_vs_c57bl6j_de_peaks_fc_2_anno_muts.txt    0.565
balbcj_vs_c57bl6j_de_peaks_fc_4_anno_muts.txt    0.588
balbcj_vs_c57bl6j_nonsig_anno_muts.txt           0.161
dtype: float64

This can be put into a fancy table for publication